# Model Building - Neural Networks

This notebook implements the model building for my thesis comparing different supervised machine learning techniques (logistic regression, random forests, support vector machines and neural networks). The main focus surrounds when each method results in better performance as measured by discrimination (AUC) and calibration (calibration plots). This project considers various factors including the number of features, number of datapoints and 'predictor strength'. This notebook fits the neural network to a small number of variables and outputs prediction and results to a pandas dataframe, where model evaliation and results will be undertaken in a following notebook.

In [1]:
import glob
import numpy as np
import os
import pandas as pd
import sklearn
import tensorflow as tf

from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, train_test_split
from scipy.stats import loguniform, uniform, randint

from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.random.seed(21)
tf.random.set_seed(21)

In [2]:
def hyperparameter_randomiser_nn(al_range, 
                                 dropout_rate_range,
                                 internal_nodes_range,
                                 learning_rate_range,
                                 prints = False):
    '''Randomises hyperparameters from a given list:
       al_range: The upper and lower bounds of the number of hidden layers
       dropout_rate_range: A list of dropout rates randomly chosen in each hidden layer
       internal_nodes_range: The upper and lower bounds for the number of internal nodes in each hidden layer
       learning_rate_range: The upper and lower bounds for the learning rate
       '''
    no_of_layers = randint(al_range[0], al_range[1] + 1).rvs(1).item()
    int_nodes = [randint(internal_nodes_range[0], internal_nodes_range[1] + 1).rvs(1).item() for i in range(no_of_layers)]
    dropout_rate = [np.random.choice(dropout_rate_range) for i in range(no_of_layers)]
    learning_rate = loguniform(learning_rate_range[0], learning_rate_range[1]).rvs(1).item()
    if prints == True:
        print("n_layers:", no_of_layers)
        print("learning_rate:", learning_rate)
        for i in range(no_of_layers):
            print(f"int_nodes_{i}:", int_nodes[i])
            print(f"dropout_rate_{i}:", dropout_rate[i])
    return no_of_layers, dropout_rate, int_nodes, learning_rate

In [3]:
def build_model_nn(no_of_layers, dropout_rate, int_nodes, learning_rate, feat_count):
    '''Builds the model based on hyperparameters provided
       no_of_layers: no of hidden layers in the model (must be greater than one)                    
       dropout_rate: The dropout rate in each hidden layer
       int_nodes: The number of internal nodes in each hidden layer
       learning_rate: The learning rate
       feat_count: number of input features
       '''
    #building architecture
    model = Sequential()
    #first_hidden_layer
    model.add(Dense(int_nodes[0], input_shape = (feat_count,), activation = 'relu'))
    model.add(Dropout(rate = dropout_rate[0]))
    #additional layers
    for i in range(no_of_layers - 1):
        model.add(Dense(int_nodes[i + 1], activation = 'relu'))
        model.add(Dropout(rate = dropout_rate[i + 1]))
    #output layer
    model.add(Dense(1, activation = 'sigmoid'))
    #compiling model
    model.compile(optimizer = Adam(learning_rate = learning_rate), 
                  loss = "binary_crossentropy", 
                  metrics = ["AUC"])
    return model

In [4]:
def hyp_tuner_nn(al_range, 
                 dr_range, 
                 lr_range, 
                 in_range,  
                 X_train, 
                 y_train, 
                 X_test, 
                 y_test,
                 cv_folds = 5, 
                 random_search = 20):
    '''Returns the best hyperparameters from given ranges using a random search algorithm:
        al_range: Range of hidden layers
        dr_range: Range of dropout layers
        lr_range: Range of learning rate
        in_range: Range of number of internal nodes in a hidden layer
        X_train: The training dataset (as a pandas dataframe) features
        y_train: The training dataset (as a pandas dataframe) labels
        X_test: The testing dataset (as a pandas dataframe) features
        y_test: The testing dataset (as a pandas dataframe) labels
        cv_folds: Number of folds of cross validation (default 5)
        random_search: how many random searches are performed (default 20)
        '''
    cv = StratifiedKFold(n_splits = cv_folds, shuffle = True, random_state = 21)
    cv.get_n_splits(X_train, y_train)
    #splitting the dataset for k-fold
    nl_list, dr_list, lr_list, in_list, ep_list = [], [], [], [], []
    auc_dict = dict()
    #creating dictionaries and lists that store information on hyperparameters and aucs
    pred_dict = dict()
    for i in range(random_search):
        print("Running search {counter} out of {maxi}".format(counter = str(i+1), maxi = str(random_search)))
        #randomly selecting hyperparameters and storing them
        n_layers, dr_rate, int_nodes, learning_rate = hyperparameter_randomiser_nn(al_range = al_range,
                                                                                   dropout_rate_range = dr_range,
                                                                                   learning_rate_range = lr_range,
                                                                                   internal_nodes_range = in_range)
        nl_list.append(n_layers)
        dr_list.append(dr_rate)
        in_list.append(int_nodes)
        lr_list.append(learning_rate)
        auc_dict[i] = []
        epochs = []
        for train_index, test_index in cv.split(X_train, y_train):
            #cross validating
            model = build_model_nn(no_of_layers = n_layers, 
                                   dropout_rate = dr_rate, 
                                   int_nodes = int_nodes, 
                                   learning_rate = learning_rate, 
                                   feat_count = X_train.shape[1])
            X1_train, X1_valid = X_train.iloc[train_index], X_train.iloc[test_index]
            y1_train, y1_valid = y_train.iloc[train_index], y_train.iloc[test_index]
            hist = model.fit(X1_train.values, 
                             y1_train.values, 
                             batch_size = 16,
                             shuffle = True,
                             validation_data = (X1_valid, y1_valid),
                             epochs = 30,
                             verbose = 0,
                             callbacks = EarlyStopping(monitor = 'val_loss', patience = 1, restore_best_weights = True))
            epochs.append(len(hist.history['loss']))
            pred = model.predict(x = X1_valid, batch_size = 16)
            auc_dict[i].append(roc_auc_score(y1_valid, pred)) 
        epochs = max(epochs)
        ep_list.append(epochs)
            
    #analysing results to find best hyperparameters
    hp_dict = {"hidden_layers": nl_list,
               "dropout_rates": dr_list,
               "internal_nodes": in_list,
               "learning_rate": lr_list,
               "epochs": ep_list}
    av_auc_dict = dict()
    for k, v in auc_dict.items():
        av_auc_dict[k] = sum(v)/len(v)
    opt_hyper = max(av_auc_dict, key = av_auc_dict.get)
    opt_layers = hp_dict["hidden_layers"][opt_hyper]
    opt_dropout = hp_dict["dropout_rates"][opt_hyper]
    opt_internal =  hp_dict["internal_nodes"][opt_hyper]
    opt_learning = hp_dict["learning_rate"][opt_hyper]
    opt_epoch = hp_dict["epochs"][opt_hyper]
    #building the model based on the whole training data based on optimal parameters
    opt_model = build_model_nn(no_of_layers = opt_layers,
                               dropout_rate = opt_dropout,
                               int_nodes = opt_internal,
                               learning_rate = opt_learning,
                               feat_count = X_train.shape[1])
    opt_model.fit(X_train.values,
                  y_train.values,
                  batch_size = 16,
                  shuffle = True, 
                  epochs = opt_epoch,
                  verbose = 0)
    predictions = opt_model.predict(x = X_test, batch_size = 16)
    v_auc = roc_auc_score(y_test, predictions)
    #sorting prediction df
    y_test_df = pd.DataFrame(y_test)
    pred_df = pd.DataFrame(predictions)
    pred_df['index'] = y_test.index
    pred_df.set_index('index', inplace = True)
    pred_df.rename(columns = {0: "prob"}, inplace = True)
    df = pred_df.join(y_test_df, how = 'left')
    print("The best hyperparameters were:")
    print("n_layers:", opt_layers)
    print("learning_rate:", opt_learning)
    for j in range(opt_layers):
        print(f"int_nodes_{j+1}:", opt_internal[j])
        print(f"dropout_rate_{j+1}:", opt_dropout[j])
    
    return df, v_auc

In [5]:
def min_max_scaling(df):
    # copy the dataframe
    df_norm = df.copy()
    # apply min-max scaling
    for column in df_norm.columns:
        df_norm[column] = (df_norm[column] - df_norm[column].min()) / (df_norm[column].max() - df_norm[column].min())
        
    return df_norm
# credit:
# https://towardsdatascience.com/data-normalization-with-pandas-and-scikit-learn-7c1cc6ed6475#:~:text=maximum%20absolute%20scaling.-,The%20min%2Dmax%20feature%20scaling,max()%20methods.

In [6]:
current_wd = os.getcwd()
base_wd = current_wd
base_wd = base_wd.replace("\\", "/")
path = base_wd + "/data/bootstraps"
os.chdir(path)
all_filenames = [i for i in glob.glob('*.{}'.format('csv'))]

In [7]:
%%time
#fitting neural networks to each bootstrap
np.random.seed(21)
i = 0
dataframes = []
for filename in all_filenames:
    i += 1
    print(str(i) + " of " + str(len(all_filenames)))
    mf = pd.read_csv(filename)
    df = min_max_scaling(mf)
    X_train, X_test, y_train, y_test = train_test_split(df.drop('deceased', axis = 1), 
                                                        df['deceased'], test_size = 0.2, 
                                                        random_state = 21, 
                                                        stratify = df['deceased'])
    preds, nn_auc = hyp_tuner_nn(al_range = [1, 5],
                                 dr_range = [i/10 for i in range (4)],
                                 lr_range = [0.0001, 0.1],
                                 in_range = [36, 360],
                                 X_train = X_train,
                                 y_train = y_train,
                                 X_test = X_test,
                                 y_test = y_test,
                                 cv_folds = 3,
                                 random_search = 5)
    preds.reset_index(drop = True, inplace = True)
    dataframes.append(preds)
    print(nn_auc)
nn_predictions = pd.concat(dataframes, axis = 1)
nn_predictions.to_csv('./predictions/nn_predictions.csv', index = False)

1 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.00012836393729688728
int_nodes_1: 243
dropout_rate_1: 0.0
int_nodes_2: 348
dropout_rate_2: 0.0
0.8770890002386065
2 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.002951866247087919
int_nodes_1: 243
dropout_rate_1: 0.2
int_nodes_2: 217
dropout_rate_2: 0.2
0.8564214152953443
3 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 4
learning_rate: 0.0009621100943436009
int_nodes_1: 54
dropout_rate_1: 0.0
int_nodes_2: 331
dropout_rate_2: 0.0
int_nodes_3: 281
dropout_rate_3: 0.2
int_nodes_4: 290
dropout_rate_4: 0.2
0.87616107

The best hyperparameters were:
n_layers: 3
learning_rate: 0.0020169815366780745
int_nodes_1: 106
dropout_rate_1: 0.0
int_nodes_2: 211
dropout_rate_2: 0.0
int_nodes_3: 177
dropout_rate_3: 0.2
0.8891111591695502
26 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.0016619614125168174
int_nodes_1: 262
dropout_rate_1: 0.2
0.8886548410938655
27 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.0009440108544895371
int_nodes_1: 229
dropout_rate_1: 0.1
0.8823928339314475
28 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.0006658613595301314
int_nodes_1: 162
d

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 5
learning_rate: 0.0001924346341798321
int_nodes_1: 259
dropout_rate_1: 0.3
int_nodes_2: 117
dropout_rate_2: 0.0
int_nodes_3: 285
dropout_rate_3: 0.0
int_nodes_4: 284
dropout_rate_4: 0.0
int_nodes_5: 306
dropout_rate_5: 0.1
0.8882419969376492
51 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 3
learning_rate: 0.005232880021086129
int_nodes_1: 238
dropout_rate_1: 0.0
int_nodes_2: 46
dropout_rate_2: 0.2
int_nodes_3: 144
dropout_rate_3: 0.3
0.8656267955202652
52 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.020312733925301012
int_nodes_1: 262
dropout_rate_1: 0.1
int_node

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 5
learning_rate: 0.002755486801798542
int_nodes_1: 182
dropout_rate_1: 0.0
int_nodes_2: 163
dropout_rate_2: 0.3
int_nodes_3: 115
dropout_rate_3: 0.2
int_nodes_4: 198
dropout_rate_4: 0.1
int_nodes_5: 120
dropout_rate_5: 0.2
0.8901091510050347
75 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.0016773677952367483
int_nodes_1: 313
dropout_rate_1: 0.2
int_nodes_2: 166
dropout_rate_2: 0.3
0.8801219886379484
76 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 3
learning_rate: 0.0005288582887178151
int_nodes_1: 87
dropout_rate_1: 0.0
int_nodes_2: 198
dropout_rate_2: 0.1
int_nod

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.00881244284455272
int_nodes_1: 118
dropout_rate_1: 0.0
0.8651370011397137
100 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 4
learning_rate: 0.0003732491884435162
int_nodes_1: 161
dropout_rate_1: 0.1
int_nodes_2: 340
dropout_rate_2: 0.3
int_nodes_3: 189
dropout_rate_3: 0.0
int_nodes_4: 218
dropout_rate_4: 0.0
0.861528816443509
101 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.0001807986334512652
int_nodes_1: 264
dropout_rate_1: 0.1
0.8895879526977086
102 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running s

...

Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.030729412635036563
int_nodes_1: 355
dropout_rate_1: 0.2
0.871638803969631
899 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.010669113202977759
int_nodes_1: 136
dropout_rate_1: 0.0
0.9055623555011562
900 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 3
learning_rate: 0.0008825775516534292
int_nodes_1: 90
dropout_rate_1: 0.0
int_nodes_2: 215
dropout_rate_2: 0.1
int_nodes_3: 121
dropout_rate_3: 0.3
0.8803418803418802
901 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
lear

The best hyperparameters were:
n_layers: 2
learning_rate: 0.011388014905240301
int_nodes_1: 165
dropout_rate_1: 0.2
int_nodes_2: 105
dropout_rate_2: 0.1
0.895175375805297
924 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.014118147089485448
int_nodes_1: 248
dropout_rate_1: 0.2
0.8888761632068719
925 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.015870083084328808
int_nodes_1: 292
dropout_rate_1: 0.0
0.869707652316348
926 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 4
learning_rate: 0.0160963368613123
int_nodes_1: 357
dropout_rate_1: 0.1
int_nodes_2: 82
dropout

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.0027551123782749705
int_nodes_1: 139
dropout_rate_1: 0.1
0.8857699805068225
950 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 3
learning_rate: 0.000571168087127436
int_nodes_1: 92
dropout_rate_1: 0.3
int_nodes_2: 357
dropout_rate_2: 0.2
int_nodes_3: 356
dropout_rate_3: 0.3
0.8675569081162386
951 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.0010488733266063527
int_nodes_1: 98
dropout_rate_1: 0.3
int_nodes_2: 89
dropout_rate_2: 0.0
0.8817374010144874
952 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running se

Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.0027456698119682757
int_nodes_1: 330
dropout_rate_1: 0.3
0.8608730001975113
973 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 5
learning_rate: 0.0010467304606456838
int_nodes_1: 266
dropout_rate_1: 0.3
int_nodes_2: 149
dropout_rate_2: 0.3
int_nodes_3: 309
dropout_rate_3: 0.3
int_nodes_4: 183
dropout_rate_4: 0.1
int_nodes_5: 250
dropout_rate_5: 0.0
0.8838199513381996
974 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.00833982068410971
int_nodes_1: 183
dropout_rate_1: 0.2
0.8974017443345619
975 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of

Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 2
learning_rate: 0.001077160405051721
int_nodes_1: 192
dropout_rate_1: 0.0
int_nodes_2: 193
dropout_rate_2: 0.2
0.9047870673055395
998 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 5
learning_rate: 0.001700838082666457
int_nodes_1: 63
dropout_rate_1: 0.3
int_nodes_2: 229
dropout_rate_2: 0.3
int_nodes_3: 143
dropout_rate_3: 0.3
int_nodes_4: 340
dropout_rate_4: 0.1
int_nodes_5: 175
dropout_rate_5: 0.0
0.8734016999260902
999 of 999
Running search 1 out of 5
Running search 2 out of 5
Running search 3 out of 5
Running search 4 out of 5
Running search 5 out of 5
The best hyperparameters were:
n_layers: 1
learning_rate: 0.0015927770918435023
int_nodes_1: 84
dropout_rate_1: 0.0
0.8824224605096275
Wall time: 7h 21min 15s
